# Perf Tools Analisys

Neste notebook iremos verificar as analises referentes ao perf tools da implementação do loop fusion, verificando quanto de acesso ao cache misses e cache references é utilizado.

In [73]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime
import time
from IPython.display import display
from collections import Counter
import os
import re
import math
import random
pd.set_option('display.max_rows', 300)
pd.options.display.float_format = '{:,.2f}'.format

In [74]:
#loads the csv as a dataframe and standardizes the algorithm names 
def load_df(filename):
    df = pd.read_csv(filename, index_col=False)
    return select_columns_and_rename_values(df)

In [75]:
#filters by substring (there are multiple OzaBag algorithms)
def filter_by_substring_algorithm(df, string):
    aux = df[df['algorithm'].str.contains(string, regex=False)]
    ret = aux
    if string == 'OB':
        ret = aux[~aux.algorithm.str.contains("Adwin|ASHT")]
    elif string == 'OzaBag':
        ret = aux[(aux.algorithm.str.contains(string)) & (~aux.algorithm.str.contains("Adwin|ASHT"))]
    return ret

In [76]:
#standardize algorithm names
def select_columns_and_rename_values(df):
    df.algorithm = df.algorithm.str.replace("Executor", "")
    df['algorithm'] = df["algorithm"].str.replace("OzaBag", "OB")
    df['algorithm'] = df["algorithm"].str.replace("AdaptiveRandomForest", "ARF")
    df['algorithm'] = df["algorithm"].str.replace("SequentialChunk", "SeqMB")
    df['algorithm'] = df["algorithm"].str.replace("OB$", "OBSequential")
    df['algorithm'] = df['algorithm'].str.replace("LeveragingBag", "LBagSequential")
    df['algorithm'] = df['algorithm'].str.replace("Adwin$", "AdwinSequential")
    df['algorithm'] = df['algorithm'].str.replace("CHUNK", "MB")
    df['algorithm'] = df['algorithm'].str.replace("MAXChunk", "MB")
    df['algorithm'] = df['algorithm'].str.replace("StreamingRandomPatches", "SRP")
    df['algorithm'] = df['algorithm'].str.replace("SRP$", "SRPSequential")
    df['algorithm'] = df['algorithm'].str.replace("OBASHT$", "OBASHTSequential")
    df.batch_size.unique()
    return df

In [77]:
import re
MILLION = 1000000
def parse_perf_file(fname, methodology='MB'):
    spname = fname.split('/')[-1].split('-')
    
    cores = 4
    if methodology == 'Sequential':
        cores = 1

    lineDict = {
        'dataset': spname[1],
        'algorithm': spname[2],
        'ensemble': spname[3],
        'methodology': methodology,
        'cores': cores,
        'batch_size': spname[5],
        'rate': spname[6],
        'cache-misses': None,
        'cache-references': None
    }

    with open (fname) as file:
        cache_misses = "0"
        cache_references = "0"
        for line in file:
            line = re.findall(r'\S+', line)
            try:
                if line[1]:
                    if line[1] == "cache-misses:u":
                        cache_misses = line[0]
                        
                    if line[1] == "cache-references:u":
                        cache_references = line[0]
            except IndexError:
                pass
    
    lineDict['cache-misses'] = float(cache_misses.replace(',', '')) / MILLION
    lineDict['cache-references'] = float(cache_references.replace(',', '')) / MILLION
            
    return lineDict

In [78]:
#Folder inside results directory that contains all the MOA dump files for these experiments
%cd /Users/reginaldoluisdeluna/Documents/Ufscar/comparison-xue3m-minibatching/
folderMOADumps = "/Users/reginaldoluisdeluna/Documents/Ufscar/comparison-xue3m-minibatching/results/analise-perf/without-loop-fusion/first"
wantedCSVfilename = "without-loop-fusion-perf"

/Users/reginaldoluisdeluna/Documents/Ufscar/comparison-xue3m-minibatching


In [79]:
resultsFolder = f"{folderMOADumps}"
csvFile = f"parsed_csvs/{wantedCSVfilename}"
directory = os.fsencode(resultsFolder)
header_printed = False

response_mb = []
with open(f"{csvFile}", "w+") as output:
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.startswith("perf-"):
            response_mb.append(
                parse_perf_file(f'{os.fsdecode(directory)}/{filename}', 'MB')
            )

df_mini_batching = pd.DataFrame.from_dict(response_mb)
display(df_mini_batching.sort_values(by=['dataset', 'algorithm']).drop_duplicates())

,dataset,algorithm,ensemble,methodology,cores,batch_size,rate,cache-misses,cache-references
16,GMSC,AdaptiveRandomForestExecutorMAXChunk,25,MB,4,50,1,"2,027.89","76,135.73"
5,GMSC,LBagExecutorMAXChunk,25,MB,4,50,1,"1,486.54","56,108.91"
1,GMSC,OzaBagASHTExecutorMAXChunk,25,MB,4,50,1,606.80,"18,773.45"
10,GMSC,OzaBagAdwinExecutorMAXChunk,25,MB,4,50,1,560.70,"22,995.69"
19,GMSC,OzaBagExecutorMAXChunk,25,MB,4,50,1,636.52,"18,302.95"
15,GMSC,StreamingRandomPatchesExecutorMAXChunk,25,MB,4,50,1,"3,839.67","146,558.32"
18,airlines,AdaptiveRandomForestExecutorMAXChunk,25,MB,4,50,1,"1,382.60","54,689.00"
2,airlines,LBagExecutorMAXChunk,25,MB,4,50,1,"1,648.24","64,972.79"
17,airlines,OzaBagASHTExecutorMAXChunk,25,MB,4,50,1,652.37,"22,129.59"
9,airlines,OzaBagAdwinExecutorMAXChunk,25,MB,4,50,1,585.58,"20,861.01"


In [82]:
#Folder inside results directory that contains all the MOA dump files for these experiments
%cd /Users/reginaldoluisdeluna/Documents/Ufscar/comparison-xue3m-minibatching/
folderMOADumps = "/Users/reginaldoluisdeluna/Documents/Ufscar/comparison-xue3m-minibatching/results/analise-perf/loop-fusion/first"
wantedCSVfilename = "loop-fusion-perf"

/Users/reginaldoluisdeluna/Documents/Ufscar/comparison-xue3m-minibatching


In [83]:
resultsFolder = f"{folderMOADumps}"
csvFile = f"parsed_csvs/{wantedCSVfilename}"
directory = os.fsencode(resultsFolder)
header_printed = False

response = []
with open(f"{csvFile}", "w+") as output:
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.startswith("perf-"):
            response.append(
                parse_perf_file(f'{os.fsdecode(directory)}/{filename}', 'MB-LF')
            )

df = pd.DataFrame.from_dict(response)
display(df.sort_values(by=['dataset', 'algorithm']).drop_duplicates())

,dataset,algorithm,ensemble,methodology,cores,batch_size,rate,cache-misses,cache-references
16,GMSC,AdaptiveRandomForestExecutorMAXChunk,25,MB-LF,4,50,1,"1,887.24","73,105.26"
5,GMSC,LBagExecutorMAXChunk,25,MB-LF,4,50,1,"1,383.87","57,531.74"
1,GMSC,OzaBagASHTExecutorMAXChunk,25,MB-LF,4,50,1,367.46,"12,128.63"
10,GMSC,OzaBagAdwinExecutorMAXChunk,25,MB-LF,4,50,1,542.18,"23,920.47"
19,GMSC,OzaBagExecutorMAXChunk,25,MB-LF,4,50,1,385.37,"12,346.34"
15,GMSC,StreamingRandomPatchesExecutorMAXChunk,25,MB-LF,4,50,1,"3,786.70","146,360.10"
18,airlines,AdaptiveRandomForestExecutorMAXChunk,25,MB-LF,4,50,1,"1,363.72","55,035.64"
2,airlines,LBagExecutorMAXChunk,25,MB-LF,4,50,1,"1,674.82","66,511.35"
17,airlines,OzaBagASHTExecutorMAXChunk,25,MB-LF,4,50,1,286.97,"8,747.20"
9,airlines,OzaBagAdwinExecutorMAXChunk,25,MB-LF,4,50,1,588.22,"21,441.84"


In [86]:
#Folder inside results directory that contains all the MOA dump files for these experiments
%cd /Users/reginaldoluisdeluna/Documents/Ufscar/comparison-xue3m-minibatching/
folderMOADumps = "/Users/reginaldoluisdeluna/Documents/Ufscar/comparison-xue3m-minibatching/results/analise-perf/loop-fusion-sequential/first"
wantedCSVfilename = "sequential-perf"

/Users/reginaldoluisdeluna/Documents/Ufscar/comparison-xue3m-minibatching


In [88]:
resultsFolder = f"{folderMOADumps}"
csvFile = f"parsed_csvs/{wantedCSVfilename}"
directory = os.fsencode(resultsFolder)
header_printed = False

response = []
with open(f"{csvFile}", "w+") as output:
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.startswith("perf-"):
            response.append(
                parse_perf_file(f'{os.fsdecode(directory)}/{filename}', 'Sequential')
            )

df = pd.DataFrame.from_dict(response)
display(df.sort_values(by=['dataset', 'algorithm']).drop_duplicates())

,dataset,algorithm,ensemble,methodology,cores,batch_size,rate,cache-misses,cache-references
20,GMSC,AdaptiveRandomForestSequential,25,Sequential,1,1,1,"2,152.19","75,131.02"
6,GMSC,LeveragingBag,25,Sequential,1,1,1,"1,597.05","51,179.12"
7,GMSC,OzaBag,25,Sequential,1,1,1,710.65,"17,018.17"
13,GMSC,OzaBagASHT,25,Sequential,1,1,1,705.63,"17,915.78"
18,GMSC,OzaBagAdwin,25,Sequential,1,1,1,813.90,"21,500.67"
14,GMSC,StreamingRandomPatches,25,Sequential,1,1,1,"4,375.78","160,499.93"
10,airlines,AdaptiveRandomForestSequential,25,Sequential,1,1,1,"1,331.48","55,088.72"
17,airlines,LeveragingBag,25,Sequential,1,1,1,"1,695.70","67,589.08"
0,airlines,OzaBag,25,Sequential,1,1,1,629.85,"22,055.93"
2,airlines,OzaBagASHT,25,Sequential,1,1,1,611.22,"21,771.79"
